**Tic Tac Toe**

### Tic-tac-toe Definition

> Tic-tac-toe is a simple game in which two players take turns to write Os or Xs in a set of nine squares. The first player to complete a row of three Os or three Xs is the winner. [Oxford Dictionary](https://www.oxfordlearnersdictionaries.com/us/definition/english/tic-tac-toe?q=tic+tac+toe)


TicTacToe Board with playing positions:

                         |     |
                      0  |  1  |  2
                    _____|_____|_____
                         |     |
                      3  |  4  |  5
                    _____|_____|_____
                         |     |
                      6  |  7  |  8
                         |     |
             
Winning combinations:

        [0, 1, 2], [3, 4, 5], [6, 7, 8],  // Rows       
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  // Cols
        [0, 4, 8], [6, 4, 2]              // Diags

### Requirments

#### Create a Game
1. To create a game, the player need to send 1 ether and select a mark (X or O)
2. Game creator joins the game as the first player automatically (contract owner cannot join the game)

#### Join a Game
3. For a second player to join an existing game, need to send 1 ether.
4. Can't join a non-existing game.
5. Can't join a game that has already been started or been over
6. Second player will be automatically assigned to X if player 1 selcts O and vice versa.

#### Play
7. Once the second player joins, the game starts
8. First player plays first
9. Each player will have two minutes to make a move; or the game will be over (loses).

#### End Game
10. The game is over when a winner can be found, or the game draws
11. If there is a winner, transfer 1.8 ether to winner's address
12. If the game ended in draw, return 0.9 for each players
13. Contract owner keeps 0.1 ether from each player

### Imports

In [76]:
import os
from solcx import install_solc, set_solc_version, set_solc_version, compile_source, compile_files
from web3 import Web3, EthereumTesterProvider

from eth_tester.exceptions import TransactionFailed
from eth_tester import EthereumTester

In [77]:
install_solc('0.7.6')
set_solc_version('0.7.6')

Define python function to be used to print the grid board.

In [78]:
# Print tic tac toe board from array passed
def printBoard(strGrid):
    cells = strGrid.split(',')
    
    print("\n")
    print("\t     |     |")
    print("\t  {}  |  {}  |  {}".format(cells[0], cells[1], cells[2]))
    print('\t_____|_____|_____')
 
    print("\t     |     |")
    print("\t  {}  |  {}  |  {}".format(cells[3], cells[4], cells[5]))
    print('\t_____|_____|_____')
 
    print("\t     |     |")
 
    print("\t  {}  |  {}  |  {}".format(cells[6], cells[7], cells[8]))
    print("\t     |     |")
    print("\n")

Intialize `web3` and use the `EthreumTesterProvider` for our backend:

In [79]:
TESTER = EthereumTester()
provider = Web3.EthereumTesterProvider(ethereum_tester=TESTER)
w3 = Web3(provider)

Define three addresses to be used for testing the game:

In [80]:
GAME_CONTRACT_ACCOUNT = w3.eth.accounts[0]
PLAYER_1_ACCOUNT = w3.eth.accounts[1]
PLAYER_2_ACCOUNT = w3.eth.accounts[2]

Compiler the solidity contract:

In [81]:
source_file_name = os.path.join(os.getcwd(), 'contracts', 'TicTacToe.sol')
compiled_sol = compile_files([source_file_name]) 
contract_compiled = compiled_sol[source_file_name + ":" + "TicTacToe"]

Get the contract interface:

In [82]:
# create the Game contract interface
contract_interface = w3.eth.contract(abi=contract_compiled['abi'], bytecode=contract_compiled['bin'])
# call contract constructor to initialize
tx_hash = contract_interface.constructor().transact({'from': GAME_CONTRACT_ACCOUNT})

Get contract receipt and interface:

In [83]:
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [84]:
ttt_contract = contract_interface(address = tx_receipt['contractAddress'])

### Simple Game

Create a game with `PLAYER_1_ACCOUNT` as the creator:

In [85]:
tx_dict_1 = { 'from': PLAYER_1_ACCOUNT, 'value': w3.toWei(1, 'ether') }
try:
    ttt_contract.functions.createGame("X").transact(tx_dict_1) 
except TransactionFailed as err:
    print(err) 

Print board for the game returned by the contract:

In [86]:
gridStr = ttt_contract.functions.getCurrentGrid(PLAYER_1_ACCOUNT).call()
printBoard(gridStr)



	     |     |
	  -  |  -  |  -
	_____|_____|_____
	     |     |
	  -  |  -  |  -
	_____|_____|_____
	     |     |
	  -  |  -  |  -
	     |     |




Check contract balance, should be 1 ether added by `PLAYER_1_ACCOUNT`

In [87]:
wei_bal = ttt_contract.functions.balanceOfContract().call()
eth_amount = w3.fromWei(wei_bal, 'ether')
eth_amount

Decimal('1')

Join `PLAYER_2_ACCOUNT` to `PLAYER_1_ACCOUNT` Game

In [88]:
tx_dict_2 = { 'from': PLAYER_2_ACCOUNT, 'value': w3.toWei(1, 'ether') }
try:
    ttt_contract.functions.joinGame(PLAYER_1_ACCOUNT).transact(tx_dict_2) 
except TransactionFailed as err:
    print(err) 

Check contract balance, should be 2 ether added by `PLAYER_1_ACCOUNT`

In [89]:
wei_bal = ttt_contract.functions.balanceOfContract().call()
eth_amount = w3.fromWei(wei_bal, 'ether')
eth_amount

Decimal('2')

Make player 1 play position 0

In [90]:
try:
    ttt_contract.functions.play(PLAYER_1_ACCOUNT, 0).transact({'from': PLAYER_1_ACCOUNT}) # player 1 plays position 0
except TransactionFailed as err:
    print(err) 

Show current board

In [91]:
gridStr = ttt_contract.functions.getCurrentGrid(PLAYER_1_ACCOUNT).call()
printBoard(gridStr)



	     |     |
	  X  |  -  |  -
	_____|_____|_____
	     |     |
	  -  |  -  |  -
	_____|_____|_____
	     |     |
	  -  |  -  |  -
	     |     |




Make player 2 play position 3

In [92]:
try:
    ttt_contract.functions.play(PLAYER_1_ACCOUNT, 3).transact({'from': PLAYER_2_ACCOUNT}) # player 2 plays position 3
except TransactionFailed as err:
    print(err) 

Show current board

In [93]:
gridStr = ttt_contract.functions.getCurrentGrid(PLAYER_1_ACCOUNT).call()
printBoard(gridStr)



	     |     |
	  X  |  -  |  -
	_____|_____|_____
	     |     |
	  O  |  -  |  -
	_____|_____|_____
	     |     |
	  -  |  -  |  -
	     |     |




Make some moves to make player 2 win the game:

In [94]:
try:
    ttt_contract.functions.play(PLAYER_1_ACCOUNT, 7).transact({'from': PLAYER_1_ACCOUNT}) # player 1 plays position 7
    ttt_contract.functions.play(PLAYER_1_ACCOUNT, 5).transact({'from': PLAYER_2_ACCOUNT}) # player 2 plays position 5
    ttt_contract.functions.play(PLAYER_1_ACCOUNT, 2).transact({'from': PLAYER_1_ACCOUNT}) # player 1 plays position 2
    ttt_contract.functions.play(PLAYER_1_ACCOUNT, 4).transact({'from': PLAYER_2_ACCOUNT}) # player 2 plays position 4
except TransactionFailed as err:
    print(err) 

Show board with player 2 winning with combination [3, 4, 5]

In [95]:
gridStr = ttt_contract.functions.getCurrentGrid(PLAYER_1_ACCOUNT).call()
printBoard(gridStr)



	     |     |
	  X  |  -  |  X
	_____|_____|_____
	     |     |
	  O  |  O  |  O
	_____|_____|_____
	     |     |
	  -  |  X  |  -
	     |     |




Show contract balance, should 0.2 ether after trasnfering 1.8 ether to `PLAYER_2_ACCOUNT`

Get Game status

In [96]:
status = ttt_contract.functions.getStatus(PLAYER_1_ACCOUNT).call()
status

'WINNER'

Get winner:

In [97]:
winner = ttt_contract.functions.getWinner(PLAYER_1_ACCOUNT).call()
winner

'Player 2'

In [98]:
wei_bal = ttt_contract.functions.balanceOfContract().call()
eth_amount = w3.fromWei(wei_bal, 'ether')
eth_amount

Decimal('0.2')

Get player 2 ether balance, should be about 0.8 ether higher than the started balance

In [99]:
wei_bal = w3.eth.get_balance(PLAYER_2_ACCOUNT)
eth_amount = w3.fromWei(wei_bal, 'ether')
eth_amount

Decimal('1000000.799999999999640549')

Get player 1 ether balance, should be about 1 ether less than the started balance

In [101]:
wei_bal = w3.eth.get_balance(PLAYER_1_ACCOUNT)
eth_amount = w3.fromWei(wei_bal, 'ether')
eth_amount

Decimal('999998.999999999999397067')

### Testing Requirments

Try creating a game as the contract owner:

In [102]:
tx_dict_1 = { 'from': GAME_CONTRACT_ACCOUNT, 'value': w3.toWei(1, 'ether') }
try:
    ttt_contract.functions.createGame("X").transact(tx_dict_1) 
except TransactionFailed as err:
    print(err) 

execution reverted: owner cannot play


Try creating a game with the wrong mark

In [103]:
tx_dict_1 = { 'from': PLAYER_1_ACCOUNT, 'value': w3.toWei(1, 'ether') }
try:
    ttt_contract.functions.createGame("S").transact(tx_dict_1) 
except TransactionFailed as err:
    print(err) 

execution reverted: Select X or O


Try creatig a game without sending 1 ether

In [104]:
tx_dict_1 = { 'from': PLAYER_1_ACCOUNT }
try:
    ttt_contract.functions.createGame("X").transact(tx_dict_1) 
except TransactionFailed as err:
    print(err)

execution reverted: You have to pay 1 ether to create a game!


Try making game creator joining the game

In [105]:
tx_dict_2 = { 'from': PLAYER_1_ACCOUNT, 'value': w3.toWei(1, 'ether') }
try:
    ttt_contract.functions.joinGame(PLAYER_1_ACCOUNT).transact(tx_dict_2) 
except TransactionFailed as err:
    print(err) 

execution reverted: You are the creator of the game, automatically joind!


Try joining a game that was over

In [106]:
tx_dict_2 = { 'from': PLAYER_2_ACCOUNT, 'value': w3.toWei(1, 'ether') }
try:
    ttt_contract.functions.joinGame(PLAYER_1_ACCOUNT).transact(tx_dict_2) 
except TransactionFailed as err:
    print(err) 

execution reverted: Game already started or its over!


Trying making a move on game that was over

In [107]:
try:
    ttt_contract.functions.play(PLAYER_1_ACCOUNT, 3).transact({'from': PLAYER_2_ACCOUNT}) # player 2 plays position 3
except TransactionFailed as err:
    print(err) 

execution reverted: Game is not playing


Create a valid game for further testing

In [108]:
try:
    ttt_contract.functions.createGame("X").transact({ 'from': w3.eth.accounts[3], 'value': w3.toWei(1, 'ether') }) 
    ttt_contract.functions.joinGame(w3.eth.accounts[3]).transact({ 'from': w3.eth.accounts[4], 'value': w3.toWei(1, 'ether') }) 
except TransactionFailed as err:
    print(err)

Trying playing when its not your turn

In [109]:
try:
    ttt_contract.functions.play(w3.eth.accounts[3], 3).transact({'from': w3.eth.accounts[4]})
except TransactionFailed as err:
    print(err)

execution reverted: Its not your turn


Make a none-valid move

In [110]:
try:
    ttt_contract.functions.play(w3.eth.accounts[3], 12).transact({'from': w3.eth.accounts[3]})
except TransactionFailed as err:
    print(err)

execution reverted: Not a valid move!


Make some moves to make the game end draw

In [111]:
try:
    ttt_contract.functions.play(w3.eth.accounts[3], 7).transact({'from': w3.eth.accounts[3]}) 
    ttt_contract.functions.play(w3.eth.accounts[3], 5).transact({'from': w3.eth.accounts[4]}) 
    ttt_contract.functions.play(w3.eth.accounts[3], 2).transact({'from': w3.eth.accounts[3]}) 
    ttt_contract.functions.play(w3.eth.accounts[3], 4).transact({'from': w3.eth.accounts[4]}) 
    ttt_contract.functions.play(w3.eth.accounts[3], 3).transact({'from': w3.eth.accounts[3]}) 
    ttt_contract.functions.play(w3.eth.accounts[3], 6).transact({'from': w3.eth.accounts[4]}) 
    ttt_contract.functions.play(w3.eth.accounts[3], 1).transact({'from': w3.eth.accounts[3]}) 
    ttt_contract.functions.play(w3.eth.accounts[3], 0).transact({'from': w3.eth.accounts[4]}) 
    ttt_contract.functions.play(w3.eth.accounts[3], 8).transact({'from': w3.eth.accounts[3]})
except TransactionFailed as err:
    print(err) 

Print the board

In [112]:
gridStr = ttt_contract.functions.getCurrentGrid(w3.eth.accounts[3]).call()
printBoard(gridStr)



	     |     |
	  O  |  X  |  X
	_____|_____|_____
	     |     |
	  X  |  O  |  O
	_____|_____|_____
	     |     |
	  O  |  X  |  X
	     |     |




Get game status

In [114]:
status = ttt_contract.functions.getStatus(w3.eth.accounts[3]).call()
status

'Draw'

Try getting game winner

In [115]:
winner = ttt_contract.functions.getWinner(w3.eth.accounts[3]).call()
winner

'Draw'

Check player 1 balance, should be about 0.8 ether

In [116]:
wei_bal = w3.eth.get_balance(w3.eth.accounts[3])
eth_amount = w3.fromWei(wei_bal, 'ether')
eth_amount

Decimal('999999.899999999999199307')

Check player 2 balance, should be about 0.8 ether

In [117]:
wei_bal = w3.eth.get_balance(w3.eth.accounts[3])
eth_amount = w3.fromWei(wei_bal, 'ether')
eth_amount

Decimal('999999.899999999999199307')